Import necessary modules

In [18]:
from openai import OpenAI
from helpers.config import Config
import json
import requests

Initiate Open AI Client

In [19]:
client = OpenAI(api_key=Config.OPENAI_API_KEY)

Define Function Schema

In [20]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                },
                "required": ["location"],
            },
        },
    },
]

Define Function

In [21]:
def get_weather(location: str) -> str:
    api_key = "777c42660156447db5842748240110"
    result = requests.get(
        f"https://api.weatherapi.com/v1/current.json?key={api_key}&q={location}"
    )

    return result.json()

Prepare Prompt Template

In [22]:
messages = []
messages.append(
    {
        "role": "system",
        "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.",
    }
)
messages.append({"role": "user", "content": "How is the weather in Yangon?"})

Get Tool Response

In [23]:
response = client.chat.completions.create(
    model="gpt-4o-mini", messages=messages, tools=tools
)
tool_response = response.choices[0].message
messages.append(tool_response)
tool_response

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_xgOcmoh4pTCvuquS7KMsvrvq', function=Function(arguments='{"location":"Yangon, Myanmar"}', name='get_weather'), type='function')])

Invoke Function Calls and Append Tool Response To Messages

In [24]:
tool_calls = tool_response.tool_calls
if tool_calls:
    tool_call_id = tool_calls[0].id
    function_name = tool_calls[0].function.name
    function_args = json.loads(tool_calls[0].function.arguments)
    if function_name == "get_weather":
        result = get_weather(**function_args)
        messages.append(
            {
                "role": "tool",
                "tool_call_id": tool_call_id,
                "name": function_name,
                "content": json.dumps(result),
            }
        )

messages

[{'role': 'system',
  'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
 {'role': 'user', 'content': 'How is the weather in Yangon?'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_xgOcmoh4pTCvuquS7KMsvrvq', function=Function(arguments='{"location":"Yangon, Myanmar"}', name='get_weather'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_xgOcmoh4pTCvuquS7KMsvrvq',
  'name': 'get_weather',
  'content': '{"location": {"name": "Yangon", "region": "Yangon", "country": "Myanmar", "lat": 16.7833, "lon": 96.1667, "tz_id": "Asia/Yangon", "localtime_epoch": 1727945420, "localtime": "2024-10-03 15:20"}, "current": {"last_updated_epoch": 1727945100, "last_updated": "2024-10-03 15:15", "temp_c": 30.2, "temp_f": 86.4, "is_day": 1, "condition": {"text": "Partly cloudy", "icon": "//cdn.weatherapi.com/weather/64x6

Inovke LLM using Tool Response

In [25]:
response = client.chat.completions.create(model="gpt-4o-mini", messages=messages)
response.choices[0].message.content

'The current weather in Yangon, Myanmar, is as follows:\n\n- **Temperature:** 30.2°C (86.4°F)\n- **Condition:** Partly cloudy\n- **Humidity:** 79%\n- **Wind:** 11.0 mph (17.6 kph) from the south\n- **Feels like:** 37.9°C (100.3°F)\n- **Visibility:** 7.0 km (4.0 miles)\n- **Pressure:** 1006.0 mb\n\nIf you need more specific information, feel free to ask!'